# Introduction to DataFrames
**[Bogumił Kamiński](http://bogumilkaminski.pl/about/), October 5, 2022**

Let's get started by loading the `DataFrames` package.

In [1]:
using DataFrames, Random

## Constructors and conversion

### Constructors

In this section, you'll see many ways to create a `DataFrame` using the `DataFrame()` constructor.

First, we could create an empty DataFrame,

In [2]:
DataFrame()

0×0 DataFrame

Or we could call the constructor using keyword arguments to add columns to the `DataFrame`.

In [3]:
DataFrame(A=1:3, B=rand(3), C=randstring.([3,3,3]), fixed=1)

Row,A,B,C,fixed
,Int64,Float64,String,Int64
1,1,0.0473287,wXV,1
2,2,0.0133377,be7,1
3,3,0.848556,l99,1


note in column `:fixed` that scalars get automatically broadcasted.

We can create a `DataFrame` from a dictionary, in which case keys from the dictionary will be sorted to create the `DataFrame` columns.

In [4]:
x = Dict("A" => [1,2], "B" => [true, false], "C" => ['a', 'b'], "fixed" => Ref([1,1]))
DataFrame(x)

Row,A,B,C,fixed
,Int64,Bool,Char,Array…
1,1,true,a,"[1, 1]"
2,2,false,b,"[1, 1]"


This time we used `Ref` to protect a vector from being treated as a column and forcing broadcasting it into every row of `:fixed` column (note that the `[1,1]` vector is aliased in each row).

Rather than explicitly creating a dictionary first, as above, we could pass `DataFrame` arguments with the syntax of dictionary key-value pairs. 

Note that in this case, we use `Symbol`s to denote the column names and arguments are not sorted. For example, `:A`, the symbol, produces `A`, the name of the first column here:

In [5]:
DataFrame(:A => [1,2], :B => [true, false], :C => ['a', 'b'])

Row,A,B,C
,Int64,Bool,Char
1,1,true,a
2,2,false,b


Although, in general, using `Symbol`s rather than strings to denote column names is preferred (as it is faster) DataFrames.jl accepts passing strings as column names, so this also works:

In [6]:
DataFrame("A" => [1,2], "B" => [true, false], "C" => ['a', 'b'])

Row,A,B,C
,Int64,Bool,Char
1,1,true,a
2,2,false,b


You can also pass a vector of pairs, which is useful if it is constructed programatically:

In [7]:
DataFrame([:A => [1,2], :B => [true, false], :C => ['a', 'b'], :fixed => "const"])

Row,A,B,C,fixed
,Int64,Bool,Char,String
1,1,true,a,const
2,2,false,b,const


Here we create a `DataFrame` from a vector of vectors, and each vector becomes a column.

In [8]:
DataFrame([rand(3) for i in 1:3], :auto)

Row,x1,x2,x3
,Float64,Float64,Float64
1,0.105076,0.484361,0.2029
2,0.973835,0.406142,0.211313
3,0.146512,0.232444,0.481848


In [9]:
DataFrame([rand(3) for i in 1:3], [:x1, :x2, :x3])

Row,x1,x2,x3
,Float64,Float64,Float64
1,0.341714,0.822388,0.384728
2,0.357087,0.722327,0.41655
3,0.508366,0.245801,0.354742


In [10]:
DataFrame([rand(3) for i in 1:3], ["x1", "x2", "x3"])

Row,x1,x2,x3
,Float64,Float64,Float64
1,0.894928,0.673365,0.564835
2,0.169181,0.55249,0.182798
3,0.766184,0.123827,0.556198


As you can see you either pass a vector of column names as a second argument or `:auto` in which case column names are generated automatically.

In particular it is not allowed to pass a vector of scalars to `DataFrame` constructor.

In [11]:
DataFrame([1, 2, 3])

LoadError: ArgumentError: 'Vector{Int64}' iterates 'Int64' values, which doesn't satisfy the Tables.jl `AbstractRow` interface

Instead use a transposed vector if you have a vector of single values (in this way you effectively pass a two dimensional array to the constructor which is supported the same way as in vector of vectors case).

In [12]:
DataFrame(permutedims([1, 2, 3]), :auto)

Row,x1,x2,x3
,Int64,Int64,Int64
1,1,2,3


You can also pass a vector of `NamedTuple`s to construct a `DataFrame`:

In [13]:
v = [(a=1, b=2), (a=3, b=4)]
DataFrame(v)

Row,a,b
,Int64,Int64
1,1,2
2,3,4


Alternatively you can pass a `NamedTuple` of vectors:

In [14]:
n = (a=1:3, b=11:13)
DataFrame(n)

Row,a,b
,Int64,Int64
1,1,11
2,2,12
3,3,13


Here we create a `DataFrame` from a matrix,

In [15]:
DataFrame(rand(3,4), :auto)

Row,x1,x2,x3,x4
,Float64,Float64,Float64,Float64
1,0.350709,0.293933,0.767794,0.764949
2,0.305816,0.129077,0.873113,0.141421
3,0.470452,0.99309,0.690987,0.50208


and here we do the same but also pass column names.

In [16]:
DataFrame(rand(3,4), Symbol.('a':'d'))

Row,a,b,c,d
,Float64,Float64,Float64,Float64
1,0.904013,0.152628,0.0246469,0.0228231
2,0.213031,0.383676,0.188265,0.183293
3,0.991833,0.875746,0.500024,0.777708


or

In [17]:
DataFrame(rand(3,4), string.('a':'d'))

Row,a,b,c,d
,Float64,Float64,Float64,Float64
1,0.920703,0.239685,0.225095,0.266329
2,0.768013,0.621145,0.701835,0.994996
3,0.224369,0.190395,0.521458,0.492755


This is how you can create a data frame with no rows, but with predefined columns and their types:

In [18]:
DataFrame(A=Int[], B=Float64[], C=String[])

0×3 DataFrame

Finally, we can create a `DataFrame` by copying an existing `DataFrame`.

Note that `copy` also copies the vectors.

In [19]:
x = DataFrame(a=1:2, b='a':'b')
y = copy(x)
(x === y), isequal(x, y), (x.a == y.a), (x.a === y.a)

(false, true, true, false)

Calling `DataFrame` on a `DataFrame` object works like `copy`.

In [20]:
x = DataFrame(a=1:2, b='a':'b')
y = DataFrame(x)
(x === y), isequal(x, y), (x.a == y.a), (x.a === y.a)

(false, true, true, false)

You can avoid copying of columns of a data frame (if it is possible) by passing `copycols=false` keyword argument:

In [21]:
x = DataFrame(a=1:2, b='a':'b')
y = DataFrame(x, copycols=false)
(x === y), isequal(x, y), (x.a == y.a), (x.a === y.a)

(false, true, true, true)

The same rule applies to other constructors

In [22]:
a = [1, 2, 3]
df1 = DataFrame(a=a)
df2 = DataFrame(a=a, copycols=false)
df1.a === a, df2.a === a

(false, true)

You can create a similar uninitialized `DataFrame` based on an original one:

In [23]:
x = DataFrame(a=1, b=1.0)

Row,a,b
,Int64,Float64
1,1,1.0


In [24]:
similar(x)

Row,a,b
,Int64,Float64
1,10,0.0


number of rows in a new DataFrame can be passed as a second argument

In [25]:
similar(x, 0)

0×2 DataFrame

In [26]:
similar(x, 2)

Row,a,b
,Int64,Float64
1,269328704,1.33661e-315
2,269405664,8.17248e-316


You can also create a new `DataFrame` from `SubDataFrame` or `DataFrameRow` (discussed in detail later in the tutorial; in particular although `DataFrameRow` is considered a 1-dimensional object similar to a `NamedTuple` it gets converted to a 1-row `DataFrame` for convenience)

In [27]:
sdf = view(x, [1,1], :)

Row,a,b
,Int64,Float64
1,1,1.0
2,1,1.0


In [28]:
typeof(sdf)

SubDataFrame{DataFrame, DataFrames.Index, Vector{Int64}}

In [29]:
DataFrame(sdf)

Row,a,b
,Int64,Float64
1,1,1.0
2,1,1.0


In [30]:
dfr = x[1, :]

Row,a,b
,Int64,Float64
1,1,1.0


In [31]:
DataFrame(dfr)

Row,a,b
,Int64,Float64
1,1,1.0


### Conversion to a matrix

Let's start by creating a `DataFrame` with two rows and two columns.

In [32]:
x = DataFrame(x=1:2, y=["A", "B"])

Row,x,y
,Int64,String
1,1,A
2,2,B


We can create a matrix by passing this `DataFrame` to `Matrix` or `Array`.

In [33]:
Matrix(x)

2×2 Matrix{Any}:
 1  "A"
 2  "B"

In [34]:
Array(x)

2×2 Matrix{Any}:
 1  "A"
 2  "B"

This would work even if the `DataFrame` had some `missing`s:

In [35]:
x = DataFrame(x=1:2, y=[missing,"B"])

Row,x,y
,Int64,String?
1,1,missing
2,2,B


In [36]:
Matrix(x)

2×2 Matrix{Any}:
 1  missing
 2  "B"

In the two previous matrix examples, Julia created matrices with elements of type `Any`. We can see more clearly that the type of matrix is inferred when we pass, for example, a `DataFrame` of integers to `Matrix`, creating a 2D `Array` of `Int64`s:

In [37]:
x = DataFrame(x=1:2, y=3:4)

Row,x,y
,Int64,Int64
1,1,3
2,2,4


In [38]:
Matrix(x)

2×2 Matrix{Int64}:
 1  3
 2  4

In this next example, Julia correctly identifies that `Union` is needed to express the type of the resulting `Matrix` (which contains `missing`s).

In [39]:
x = DataFrame(x=1:2, y=[missing,4])

Row,x,y
,Int64,Int64?
1,1,missing
2,2,4


In [40]:
Matrix(x)

2×2 Matrix{Union{Missing, Int64}}:
 1   missing
 2  4

Note that we can't force a conversion of `missing` values to `Int`s!

In [41]:
Matrix{Int}(x)

LoadError: ArgumentError: cannot convert a DataFrame containing missing values to Matrix{Int64} (found for column y)

### Conversion to `NamedTuple` related tabular structures

First define some data frame

In [42]:
x = DataFrame(x=1:2, y=["A", "B"])

Row,x,y
,Int64,String
1,1,A
2,2,B


Now we convert a `DataFrame` into a `NamedTuple` of vectors

In [43]:
ct = Tables.columntable(x)

(x = [1, 2], y = ["A", "B"])

Next we convert it into a vector of `NamedTuples`

In [44]:
rt = Tables.rowtable(x)

2-element Vector{NamedTuple{(:x, :y), Tuple{Int64, String}}}:
 (x = 1, y = "A")
 (x = 2, y = "B")

We can perform the conversions back to a `DataFrame` using a standard constructor call:

In [45]:
DataFrame(ct)

Row,x,y
,Int64,String
1,1,A
2,2,B


In [46]:
DataFrame(rt)

Row,x,y
,Int64,String
1,1,A
2,2,B


### Iterating data frame by rows or columns

Sometimes it is useful to create a wrapper around a `DataFrame` that produces its rows or columns.

For iterating columns you can use the `eachcol` function.

In [47]:
ec = eachcol(x)

Row,x,y
,Int64,String
1,1,A
2,2,B


`DataFrameColumns` object behaves as a vector (note though it is not `AbstractVector`)

In [48]:
ec isa AbstractVector

false

In [49]:
ec[1]

2-element Vector{Int64}:
 1
 2

but you can also index into it using column names:

In [50]:
ec["x"]

2-element Vector{Int64}:
 1
 2

similarly `eachrow` creates a `DataFrameRows` object that is a vector of its rows

In [51]:
er = eachrow(x)

Row,x,y
,Int64,String
1,1,A
2,2,B


`DataFrameRows` is an `AbstractVector`

In [52]:
er isa AbstractVector

true

In [53]:
er[end]

Row,x,y
,Int64,String
2,2,B


Note that both data frame and also `DataFrameColumns` and `DataFrameRows` objects are not type stable (they do not know the types of their columns). This is useful to avoid compilation cost if you have very wide data frames with heterogenous column types.

However, often (especially if a data frame is narrows) it is useful to create a lazy iterator that produces `NamedTuple`s for each row of the `DataFrame`. Its key benefit is that it is type stable (so it is useful when you want to perform some operations in a fast way on a small subset of columns of a `DataFrame` - this strategy is often used internally by DataFrames.jl package):

In [54]:
nti = Tables.namedtupleiterator(x)

Tables.NamedTupleIterator{Tables.Schema{(:x, :y), Tuple{Int64, String}}, Tables.RowIterator{NamedTuple{(:x, :y), Tuple{Vector{Int64}, Vector{String}}}}}(Tables.RowIterator{NamedTuple{(:x, :y), Tuple{Vector{Int64}, Vector{String}}}}((x = [1, 2], y = ["A", "B"]), 2))

In [55]:
for row in enumerate(nti)
    @show row
end

row = (1, (x = 1, y = "A"))
row = (2, (x = 2, y = "B"))


similarly to the previous options you can easily convert `NamedTupleIterator` back to a `DataFrame`.

In [56]:
DataFrame(nti)

Row,x,y
,Int64,String
1,1,A
2,2,B


### Handling of duplicate column names

We can pass the `makeunique` keyword argument to allow passing duplicate names (they get deduplicated)

In [57]:
df = DataFrame(:a=>1, :a=>2, :a_1=>3; makeunique=true)

Row,a,a_2,a_1
,Int64,Int64,Int64
1,1,2,3


Otherwise, duplicates are not allowed.

In [58]:
df = DataFrame(:a=>1, :a=>2, :a_1=>3)

LoadError: ArgumentError: Duplicate variable names: :a. Pass makeunique=true to make them unique using a suffix automatically.

Observe that currently `nothing` is not printed when displaying a `DataFrame` in Jupyter Notebook:

In [59]:
df = DataFrame(x=[1, nothing], y=[nothing, "a"], z=[missing, "c"])

Row,x,y,z
,Union…,Union…,String?
1,1,,missing
2,,a,c


Finally you can use `empty` and `empty!` functions to remove all rows from a data frame:

In [60]:
empty(df)

0×3 DataFrame

In [61]:
df

Row,x,y,z
,Union…,Union…,String?
1,1,,missing
2,,a,c


In [62]:
empty!(df)

0×3 DataFrame

In [63]:
df

0×3 DataFrame